# Zad 3

Zaimplementować procedurę do tworzenia nowego czytelnika przyjmującą odpowiednie parametry. W ramach procedury należy zaimplementować następujące walidacje: poprawność formatu PESEL, nazwisko z wielkiej litery i co najmniej dwuliterowe, poprawną datę urodzenia, zgodną z PESELem. Wszystkie niezgodności należy zakomunikować poprzez odpowiednie wywołanie instrukcji THROW.

In [11]:
DROP PROCEDURE IF EXISTS createReader 
GO

CREATE PROCEDURE createReader 
    @pesel CHAR(11), 
    @nazwisko VARCHAR(30), 
    @miasto VARCHAR(30), 
    @data_Urodzenia DATE 
    -- @ostatnie_Wypozyczenie DATE = NULL
AS
BEGIN
    DECLARE @nazwisko_len INT = LEN (@nazwisko)
    DECLARE @pesel_len INT = LEN(@pesel)

    -- weryfikacja pesel
    IF (@pesel_len != 11)
         THROW 50110, 'PESEL powinien zawierać 11 znaków.', 1; 

    DECLARE @pesel_data DATE -- data wyciągnieta z PESEL'u
    DECLARE @wagi VARCHAR(11) = '13791379131'
    
    DECLARE @i INT = 1 -- tablice indeksowane od 1!
    DECLARE @char INT  -- obecnie przetwarzany znak
    DECLARE @pesel_suma INT = 0 -- suma kontrolna

    WHILE (@i <= 11)
    BEGIN
        DECLARE @c CHAR = SUBSTRING(@pesel, @i, 1)
        -- check if char is a number (using ASCII cast)
        IF (@c < '0') OR (@c > '9')
            THROW 50111, 'PESEL powinien składać się jedynie z cyfr.', 1; 

        SET @char = @c

        SET @pesel_suma += CAST(SUBSTRING(@wagi, @i, 1) as INT) * @char
        SET @i = @i + 1
    END
    
    -- PRINT @pesel_suma
    -- PRINT 'suma kontrolna: ' + CAST(@pesel_suma as VARCHAR)

    IF ((@pesel_suma % 10) != 0)
        THROW 50112, 'PESEL nie spełnia sumy kontrolnej', 1;

    DECLARE @pesel_rok INT     = SUBSTRING(@pesel, 1, 2)
    DECLARE @pesel_miesiac INT = SUBSTRING(@pesel, 3, 2)
    DECLARE @rok INT = 1900 + @pesel_rok

    -- z pierwszej cyfry miesiąca wynikają dodatkowe informacje o latach
    -- 8/9 -> 1800-1899
    -- 0/1 -> 1900-1999
    -- 2/3 -> 2000-2099
    -- 4/5 -> 2100-2199
    -- 6/7 -> 2200-2299
    DECLARE @pesel_add INT = SUBSTRING(@pesel, 3, 1)

    IF (@pesel_add >= 8)
        SET @rok -= 100
    ELSE IF (@pesel_add >= 2)
        SET @rok += (@pesel_add / 2) * 100
    
    -- miesiące po październiku wymagają operacji modulo (11, 31, 51 vs 01, 21, 41)
    DECLARE @miesiac INT = (@pesel_add % 2) * 10 + SUBSTRING(@pesel, 4, 1)

    -- konwersja intów oznaczajacych dzień, miesiąc, rok na datetime
    SET @pesel_data = CAST(
            CAST(@rok AS VARCHAR(4)) +
            RIGHT('0' + CAST(@miesiac AS VARCHAR(2)), 2) +
            SUBSTRING(@pesel, 5, 2)
        AS DATETIME)
    
    -- PRINT @pesel_data

    -- weryfikacja nazwiska
    IF (@nazwisko_len < 2)
       THROW 50113, 'Nazwisko jest za krótkie', 1;

    IF LEFT(@nazwisko, 1) != UPPER(LEFT(@nazwisko, 1)) COLLATE Latin1_General_CS_AS
        THROW 50114, 'Nazwisko powinno rozpoczynać się wielką literą.', 1;

    IF RIGHT(@nazwisko, @nazwisko_len - 1) != LOWER(RIGHT(@nazwisko, @nazwisko_len - 1)) COLLATE Latin1_General_CS_AS
        THROW 50115, 'Tylko pierwsza litera nazwiska powinna być kapitalikiem.', 1;

    -- weryfikacja daty urodzenia
    IF @pesel_data != @data_Urodzenia
        THROW 50116, 'Data urodzenia nie zgadza się z numerem PESEL.', 1;

    -- wszystkie testy przeszły pomyślnie -> dodajemy czytelnika
    INSERT INTO Czytelnik (PESEL,NAZWISKO,MIASTO,DATA_URODZENIA) VALUES
        (@pesel, @nazwisko, @miasto, @data_Urodzenia)

END

Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:00.141

Tests

In [12]:
exec createReader '55101011111', 'Kowalski', 'Wrocław', '1955-10-10'

: Msg 50112, Level 16, State 1, Procedure createReader, Line 41
PESEL nie spełnia sumy kontrolnej

Total execution time: 00:00:00.061

In [13]:
exec createReader '10', 'Kowalski', 'Wrocław', '1955-10-10'

: Msg 50110, Level 16, State 1, Procedure createReader, Line 15
PESEL powinien zawierać 11 znaków.

Total execution time: 00:00:00.046

In [14]:
exec createReader '0027283979a', 'Kowalski', 'Wrocław', '1955-10-10'

: Msg 50111, Level 16, State 1, Procedure createReader, Line 29
PESEL powinien składać się jedynie z cyfr.

Total execution time: 00:00:00.048

In [15]:
exec createReader '00272839791', 'kowalski', 'Wrocław', '1955-10-10'

: Msg 50114, Level 16, State 1, Procedure createReader, Line 77
Nazwisko powinno rozpoczynać się wielką literą.

Total execution time: 00:00:00.051

In [16]:
exec createReader '00272839791', 'KOWalski', 'Wrocław', '1955-10-10'

: Msg 50115, Level 16, State 1, Procedure createReader, Line 80
Tylko pierwsza litera nazwiska powinna być kapitalikiem.

Total execution time: 00:00:00.059

In [17]:
exec createReader '00272839791', 'K', 'Wrocław', '1955-10-10'

: Msg 50113, Level 16, State 1, Procedure createReader, Line 74
Nazwisko jest za krótkie

Total execution time: 00:00:00.054

In [18]:
exec createReader '00272839791', 'Kowalski', 'Wrocław', '1955-10-10'

: Msg 50116, Level 16, State 1, Procedure createReader, Line 84
Data urodzenia nie zgadza się z numerem PESEL.

Total execution time: 00:00:00.059

In [28]:
exec createReader '00272839791', 'Pol', 'Wrocław', '2000-07-28'

SELECT * FROM Czytelnik

(1 row affected)

(4 rows affected)

Total execution time: 00:00:00.057

Czytelnik_ID,PESEL,Nazwisko,Miasto,Data_Urodzenia,Ostatnie_Wypozyczenie
1,55101011111,Kowalski,Wroclaw,1955-10-10,NULL
2,60101033333,Maliniak,Wroclaw,1960-10-10,NULL
3,65120122222,Nowak,Warszawa,1965-12-01,NULL
19,00272839791,Pol,Wroclaw,2000-07-28,NULL


In [27]:
DELETE
FROM Czytelnik
WHERE PESEL = '00272839791'

(1 row affected)

Total execution time: 00:00:00.056